In [4]:
import os
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import cv2

# Definisci le cartelle contenenti le immagini di training e test
train_dir = '../../datasets/dataset2/train/'
test_dir = '../../datasets/dataset2/test/'

# Definisci le classi e l'etichetta numerica corrispondente
classes = ['cloudy', 'rain', 'shine', 'sunrise']
label_dict = {i: classes[i] for i in range(len(classes))}

# Carica le immagini di training
X_train = []
y_train = []
for i, cls in enumerate(classes):
    for img_file in os.listdir(train_dir + cls):
        img = cv2.imread(os.path.join(train_dir + cls + '/' + img_file))
        if img is not None:
            img_resized = cv2.resize(img, (64, 64))
            X_train.append(img_resized.flatten())
            y_train.append(i)

# Dividi il dataset di training in un subset di training e un subset di validazione
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Addestra un modello SVM lineare
clf = svm.SVC(kernel='linear', C=1.0)
clf.fit(X_train, y_train)

# Valuta le prestazioni del modello sul dataset di validazione
y_pred = clf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy on validation set: {accuracy:.4f}")

# Carica le immagini di test
X_test = []
for img_file in os.listdir(test_dir):
    img = cv2.imread(os.path.join(test_dir + cls + '/' + img_file))
    if img is not None:
        img_resized = cv2.resize(img, (64, 64))
        X_test.append(img_resized.flatten())

# Fai le previsioni sul dataset di test
y_test = clf.predict(X_test)

# Stampa le previsioni per le immagini del dataset di test
for i, y in enumerate(y_test):
    print(f"{test_dir}/{os.listdir(test_dir)[i]}: {label_dict[y]}")


Accuracy on validation set: 0.8222


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.